In [2]:
import random
import pickle
import numpy as np

# PLAYING, TRAINING AND TESTING GAME

In this cell we define the parameters

In [3]:
learning_rate=0.9 #should decrease as it continues to gain a larger knowledge base.
decay_gamma=1 #big gamma means thinking long term
exp_rate=0.2
starting_player=1
init_board='0'
n=8

file_name = 'test41'
policy_player1='poltest41'
list_of_winners=[]

In [29]:

class seq_game:
    def __init__(self, n):
        self.n=n
        self.IsEnd=False
        self.Player=starting_player
        self.board=init_board
        self.p1_states=[]
        self.exp_rate=exp_rate
        self.lr = learning_rate  # should decrease as it continues to gain a larger knowledge base
        self.decay_gamma = decay_gamma  # big gamma means thinking long term
        self.states_value = {}  # dict for storing {board, weight}
    def check_for_losing_move(self, board):
        """
        Checks whether adding 0 or 1 are losing moves
        :return returns a list of losing moves, i.e either [], [0], [1], [0,1]:
        """
        current_board=board
        list_of_losers=[]
        for i in [0,1]:
            possible_loser=current_board+str(i)
            substring_to_test=possible_loser[(-self.n):]
            if substring_to_test in current_board:
                list_of_losers.append(i)
        return list_of_losers
    def check_for_winning_move(self):
        """
        Assumes there are two non-losing moves. Checks whether one of these moves forces next player to lose
        :return:
        """
        winning_move=[]
        for i in [0,1]:
            temp_board=self.board+str(i)
            if len(self.check_for_losing_move(temp_board))==2:
                winning_move.append(i)
        return winning_move
    def change_player(self):
        """
        Function for changing current player
        :return: returns none
        """
        if self.Player==1:
            self.Player=2
        else:
            self.Player=1
    def update_board(self, move):
        """
        updates board
        :param move is a string:
        :return: returns none
        """
        self.board+=move
    def reset(self):
        """
        Resets the game
        :return: returns none
        """
        self.board = init_board
        self.IsEnd = False
        self.Player = starting_player
        self.p1_states=[]
    def savePolicy(self):
        """
        function for saving states and weights for use for the trained agent
        :return: returns None
        """
        fw = open('pol' + file_name, 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()
    def feedReward(self, reward):
        """
        function for giving rewards to all board states from the finished game.
        :param reward: reward to give. 1 or 0.
        :return: returns None
        """
        for st in reversed(self.p1_states):  # goes through all saved board states of this game
            if self.states_value.get(
                    st) is None:  # if it's not already in the dictionary (of board states of ALL games)
                self.states_value[st] = 0  # initialise a value for the state
            self.states_value[st] += self.lr * (
                        self.decay_gamma * reward - self.states_value[st])  # update weight for each board state
    def loadPolicy(self, file):
        """
        function for loading states and weights for the trained agent.
        :param file:
        :return:
        """
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()
    def smartmove(self, current_board):
        """
        function for choosing an action. This is for player 1 already trained.
            :param positions: available moves to make
            :param current_board: board state
            :return: returns action to make
         """
        losing_moves = self.check_for_losing_move(current_board)
        if len(losing_moves) == 1:  # if only one non-losing move
            non_losing_move = list({0, 1} - set(losing_moves))
            move = str(non_losing_move[0])
        elif len(losing_moves) == 0:
            winning_moves = self.check_for_winning_move()
            if len(winning_moves) > 0:
                move = str(winning_moves[0])
            else:
                value_max = -999
                #print(self.board, current_board, 'board')
                for p in [0,1]:
                    next_board = current_board+str(p)
                    value = 0 if self.states_value.get(next_board) is None else self.states_value.get(next_board)
                    if value >= value_max:
                        value_max = value
                        move = str(p)
        else:
            move= None
        return move


    def normalmove(self):
        """
        :return: returns move
        """
        losing_moves = self.check_for_losing_move(self.board)
        if len(losing_moves) == 1:  # if only one non-losing move
            non_losing_move = list({0, 1} - set(losing_moves))
            move = str(non_losing_move[0])
        elif len(losing_moves) == 0:
            winning_moves = self.check_for_winning_move()
            if len(winning_moves) > 0:
                move = str(winning_moves[0])
            else:
                move = str(random.randint(0, 1))
        else:
            move= None
        return move
    def training_game(self, rounds=10):
        for i in range(rounds):
            #if i % 1000 == 0:
            #    print("Rounds {}".format(i))
            while not self.IsEnd:
                move=self.normalmove()
                if move is not None:
                    self.update_board(move)
                else:
                    self.IsEnd = True
                    if self.Player==2:#if player 1 has won
                        self.feedReward(1)
                        self.savePolicy()
                        list_of_winners.append(1)
                    else:
                        list_of_winners.append(2)
                        self.feedReward(0)
                        self.savePolicy()
                if self.Player == 1:
                    self.p1_states.append(self.board)
                self.change_player()
            self.reset()
    def training_game2(self, rounds=10):
        for i in range(rounds):
            while not self.IsEnd:
                if self.Player==2:
                    move = self.normalmove()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.normalmove()
                    else:
                        move=self.smartmove(self.board)
                if move is not None:
                    self.update_board(str(move))
                else:
                    self.IsEnd = True
                    if self.Player==2:#if player 1 has won
                        self.feedReward(1)
                        self.savePolicy()
                        list_of_winners.append(1)
                    else:
                        list_of_winners.append(2)
                        self.feedReward(0)
                        self.savePolicy()
                if self.Player == 1:
                    self.p1_states.append(self.board)
                self.change_player()
            self.reset()
    def smart_game(self, rounds=10):
        for i in range(rounds):
            while not self.IsEnd:
                #print('PLAYER', self.Player)
                losing_moves = self.check_for_losing_move(self.board)
                if len(losing_moves) == 1:  # if only one non-losing move
                    non_losing_move = list({0, 1} - set(losing_moves))
                    move = str(non_losing_move[0])
                elif len(losing_moves) == 0:
                    winning_moves = self.check_for_winning_move()
                    if len(winning_moves) > 0:
                        move = str(winning_moves[0])
                    else:
                        """HERE IS WHERE THE EXCITING STUFF HAPPENS"""
                        if self.Player==2:
                            move = str(random.randint(0, 1))
                        else:
                            move=self.smartmove(self.board)
                self.update_board(move)
                if self.Player == 1:
                    self.p1_states.append(self.board)
                #print(self.board)
                if len(losing_moves) == 2:  # GAME OVER
                    self.IsEnd = True
                    #with open('Data', 'a') as f:  # saves winner
                    #    f.write(self.board+', ')
                    if self.Player == 2:
                        list_of_winners.append(1)
                    else:
                        list_of_winners.append(2)
                self.change_player()
            self.reset()
    def smart_game2(self, rounds=10):
        for i in range(rounds):
            while not self.IsEnd:
                if self.Player==1:
                    losing_moves = self.check_for_losing_move(self.board)
                    if len(losing_moves) == 1:  # if only one non-losing move
                        non_losing_move = list({0, 1} - set(losing_moves))
                        move = str(non_losing_move[0])
                    elif len(losing_moves) == 0:
                        winning_moves = self.check_for_winning_move()
                        if len(winning_moves) > 0:
                            move = str(winning_moves[0])
                        else:
                            move=self.smartmove(self.board)
                self.update_board(move)
                if self.Player==2:
                    losing_moves = self.check_for_losing_move(self.board)
                    if len(losing_moves) == 1:  # if only one non-losing move
                        non_losing_move = list({0, 1} - set(losing_moves))
                        move = str(non_losing_move[0])
                    if len(losing_moves)<1:
                        move=str(random.randint(0, 1))
                if len(losing_moves) == 2:  # GAME OVER
                    self.IsEnd = True
                    if self.Player == 2:
                        list_of_winners.append(1)
                    else:
                        list_of_winners.append(2)
                self.change_player()
            self.reset()
    

In [30]:
play = seq_game(n)
play.training_game(100)

263 
 01010111100100111101000100100101001110001110010101000010011010010111110111011111101011000110101110100110010110011101010010001010001111111100110110100000000100001110110000011110001000110011000011011110110111000000110001011010101011011001000001011100111110000101011


In [20]:
len('0010101000011010001001000001111011111011011110001011111111001100001110101001111110100110111001000111000111110000101100011001001100111011101100000010011100111100101101001001011100000101000110110011010101011101000000011000100001000101011011010110010101')

250

Below is for testing if a substring has repeated

In [32]:
list_of_substrings=[]
examples='01010111100100111101000100100101001110001110010101000010011010010111110111011111101011000110101110100110010110011101010010001010001111111100110110100000000100001110110000011110001000110011000011011110110111000000110001011010101011011001000001011100111110000101011'
for i in range(len(examples)-7):
    list_of_substrings.append(examples[i:i+8])
len(list_of_substrings), list_of_substrings
print(len(list_of_substrings), len(set(list_of_substrings)))


256 256


256

Below is for training until policy has more than x percent winnings

In [ ]:

i=0
while i==0:
    play = seq_game(n)
    play.training_game2(500)
    #print(list_of_winners.count(1), list_of_winners.count(2))
    list_of_winners = []
    play.loadPolicy(policy_player1)
    play.smart_game(1000)
    print(list_of_winners.count(1), list_of_winners.count(2))
    if list_of_winners.count(1)>780:
        print('JUBII')
        break

444 556
468 532
542 458
655 345
249 751
745 255
525 475
511 489
428 572
731 269
555 445
654 346
504 496
647 353
696 304
366 634
684 316
645 355
232 768
544 456
562 438
629 371
488 512
661 339
631 369
236 764
627 373
742 258
723 277
520 480
674 326
652 348
663 337
321 679
438 562
518 482
697 303
626 374
739 261
470 530
720 280
547 453
633 367
609 391
763 237
659 341
486 514
647 353
673 327
505 495
687 313
661 339
373 627
734 266
392 608
587 413
377 623
664 336
733 267
639 361
313 687
470 530
450 550
611 389
679 321
353 647
744 256
714 286
697 303
343 657
562 438
578 422
603 397
694 306
434 566
716 284
690 310
753 247
639 361
770 230
389 611
478 522
488 512
708 292
447 553
543 457
733 267
705 295
690 310
626 374
669 331
621 379
597 403
362 638
768 232
750 250
384 616
742 258
744 256
434 566
628 372
607 393
533 467
714 286
293 707
716 284
726 274
651 349
677 323
425 575
579 421
541 459
686 314
504 496
364 636
690 310
723 277
580 420
739 261
542 458
543 457
400 600
636 364
704 296
566 434


Below is for testing a certain policy file

In [10]:
play=seq_game(n)

list_of_winners=[]
play.loadPolicy('goodpolicy!')
play.smart_game(1000)
print(list_of_winners.count(1), list_of_winners.count(2))

770 230


# BELOW IS FOR ANALYZING POLICY FILES

In [136]:
fr = open('goodpolicy!', 'rb')
hej = pickle.load(fr)
print(hej['0100'], hej['0101'])
hejkey=list(iter(hej))
max_value = max(hej.values())
value = {i for i in hej if hej[i]==max_value}
#print("key by value:",value)
from collections import Counter
hej2=dict(Counter(hej).most_common(15))
#print(hej2)
#print(len(max(hejkey, key = len)))
fr.close()

0.0009908480215936573 0.0002993008995001


In [37]:
pip install 

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git
You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


SyntaxError: invalid syntax (2630282858.py, line 1)